In [11]:
import html
import io
import os
import time
from datetime import datetime
from pathlib import Path
from typing import Iterable

import requests
import yfinance as yf
from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta import Thread
from openai.types.beta.threads import Run
from openai.types.beta.threads.message_content import MessageContent
from openai.types.beta.threads.image_file_content_block import ImageFileContentBlock
from openai.types.beta.threads.text_content_block import TextContentBlock
#from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
#from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile
from PIL import Image

#### Load the environment variables

In [3]:
load_dotenv("./.env")
api_endpoint = os.getenv("OpenAiAssistantEp")
api_key = os.getenv("OpenAiAssistantKey")
api_version = os.getenv("OpenAiAssistantVersion")
api_deployment_name = os.getenv("OpenAiGpt4Turbo")

#### Create an AzureOpenAI client

In [4]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

#### Prepare the tools for function calling

In [5]:
tools_list = [
    {"type": "code_interpreter"},
]

In [9]:
dataFolder = "Data/CSV/FailedBanks/"

def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

arr = os.listdir(dataFolder)
assistant_files = []
for file in arr:
    filePath = dataFolder + file
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

#### Create an Assistant and a Thread

In [10]:
assistant = client.beta.assistants.create(
    name="Failed Banks Assistant",
    instructions="You are an assistant that can help find information about failed banks. "
    + "Use the provided file only.",
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)

thread = client.beta.threads.create()

#### Format and display the Assistant Messages for text and images

In [12]:
def read_assistant_file(file_id:str):
    response_content = client.files.content(file_id)
    return response_content.read() 

def print_messages(messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, TextContentBlock):
                print(f"{message.role}:\n{item.text.value}\n")
                file_annotations = item.text.annotations
                if file_annotations:
                    for annotation in file_annotations:
                        file_id = annotation.file_path.file_id
                        content = read_assistant_file(file_id)
                        print(f"Annotation Content:\n{str(content)}\n")
            elif isinstance(item, ImageFileContentBlock):
                # Retrieve image from file id                
                data_in_bytes = read_assistant_file(item.image_file.file_id)
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

#### Process the user messages

In [13]:
def process_prompt(prompt: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. "
        + "The current date and time is: "
        + datetime.now().strftime("%x %X")
        + ". ",
    )
    print("processing ...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            # Handle completed
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            print_messages(messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            answer = messages.data[0].content[0].text.value
            print(f"Failed User:\n{prompt}\nAssistant:\n{answer}\n")
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            print(run)
            break
        if run.status == "cancelled":
            # Handle cancelled
            print(run)
            break
        if run.status == "requires_action":
            # Handle function calling and continue processing
            pass
        else:
            time.sleep(5)

#### Process user requests

In [14]:
process_prompt("Create a chart of failed banks by state.")

processing ...
user:
Create a chart of failed banks by state.

assistant:
To create a chart of failed banks by state, I'll need to examine the contents of the file you've uploaded. I'll start by loading the data to see if it contains information about bank failures and their associated states. Then, I can proceed to analyze the data and create the appropriate chart.

Let me take a look at the file content first.

assistant:
It appears there was an error in my attempt to process the file. Let me correct this and try again to determine the content and structure of the data you provided. I will need to define the `text_data` variable correctly in the exception handling to avoid this error. Let me make this adjustment and try once more.

assistant:
The file you uploaded appears to contain a data table with information on bank failures. The columns include "Bank Name", "City", "State", "Cert" (likely certification number), "Acquiring Institution", "Closing Date", and "Fund" (which could ref

In [14]:
process_prompt("What was the last bank to fail?")

processing ...
user:
What was the last bank to fail?

assistant:
To determine the last bank to fail, I will sort the data by the "Closing Date" and find the most recent entry. I'll do this now for you.

assistant:
The last bank to fail, according to the data provided, was Citizens Bank, located in Sac City, Iowa. The bank failed on November 3, 2023.

If you have any more questions or need further assistance, please feel free to ask.



In [15]:
process_prompt("Which state has the most bank failures?")

processing ...
user:
Which state has the most bank failures?

assistant:
Based on the previously summarized data, I will identify the state with the most bank failures. Let me provide you with that information now.

assistant:
The state with the most bank failures, as per the data we have, is Georgia (GA), with a total of 93 failed banks.

Should you need any additional information or assistance, please let me know.



In [16]:
process_prompt("Which banks failed in Florida in between 2020-2023?")

processing ...
user:
Which banks failed in Florida in between 2020-2023?

assistant:
To find out which banks failed in Florida between 2020 and 2023, I will filter the data for banks located in Florida, and then further filter by the closing date to fall within the specified range from January 1, 2020, to December 31, 2023.

Let's do this now.

assistant:
Between 2020 and 2023, the following bank failed in Florida:

- First City Bank of Florida in Fort Walton Beach. The bank failed on October 16, 2020.

There are no other entries for failed banks in Florida within the specified date range in the dataset provided.

If you have any further questions or require more information, please let me know.



#### Cleanup

In [15]:
client.beta.assistants.delete(assistant.id)
client.beta.threads.delete(thread.id)
for file in assistant_files:
    client.files.delete(file.id)

#### Using Functions

###### Get the latest stock price by ticker symbol using Yahoo Finance

In [16]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    return stock.history(period="1d")["Close"].iloc[-1]

#### Define the Assistant tools

In [17]:
tools_list = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
            "parameters": {
                "type": "object",
                "properties": {"symbol": {"type": "string", "description": "The ticker symbol of the stock"}},
                "required": ["symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Sends an email to a recipient(s).",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {"type": "string", "description": "The email(s) the email should be sent to."},
                    "content": {"type": "string", "description": "The content of the email."},
                },
                "required": ["to", "content"],
            },
        },
    },
]

In [18]:
def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

filePath = 'Data/CSV/portfolio.csv'
assistant_files = []
assistant_files.append(upload_file(client, filePath))
file_ids = [file.id for file in assistant_files]

In [19]:
assistant = client.beta.assistants.create(
    name="Portfolio Management Assistant",
    instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly. "
    + "Use the provided portfolio CSV file to answer the questions. If question is not related to the portfolio or you cannot answer the question, say, 'contact a representative for more assistance.'"
    + "If the user asks for help or says 'help', provide a list of sample questions that you can answer.",
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)

In [20]:
thread = client.beta.threads.create()

In [21]:
def call_functions(client: AzureOpenAI, thread: Thread, run: Run) -> None:
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json

    for action in required_actions["tool_calls"]:
        func_name = action["function"]["name"]
        arguments = json.loads(action["function"]["arguments"])

        if func_name == "get_stock_price":
            output = get_stock_price(symbol=arguments["symbol"])
            tool_outputs.append({"tool_call_id": action["id"], "output": output})
        else:
            raise ValueError(f"Unknown function: {func_name}")

    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

In [22]:
process_prompt("Based on the provided portfolio, what investments do I own?")

processing ...
user:
Based on the provided portfolio, what investments do I own?

assistant:
To answer your question, I'll begin by examining the file you uploaded to determine the contents and identify the investments listed in your portfolio. Allow me a moment to review the file, and I'll share the details with you shortly.

assistant:
Based on the content of the file, your portfolio includes the following investments with their respective average cost and quantity:

1. Microsoft Corporation (MSFT) - Average Cost: $200, Quantity: 300 shares
2. Apple Inc. (AAPL) - Average Cost: $114, Quantity: 200 shares
3. Amazon.com, Inc. (AMZN) - Average Cost: $125, Quantity: 50 shares
4. Tesla, Inc. (TSLA) - Average Cost: $900, Quantity: 100 shares
5. Netflix, Inc. (NFLX) - Average Cost: $540, Quantity: 80 shares
6. NVIDIA Corporation (NVDA) - Average Cost: $450, Quantity: 50 shares

These are the investments reflected in your portfolio according to the uploaded file. If you require any further de

In [23]:
process_prompt("What is the value of my portfolio?")

processing ...
Run(id='run_B4wo2pdMk1B4fEiIMSyB0Mdj', assistant_id='asst_GLYqs7FKGhO6eSbAqLEcOGVZ', cancelled_at=None, completed_at=None, created_at=1714679499, expires_at=None, failed_at=None, file_ids=['assistant-CgPhDS6uXvZkiaQVRVLLK3jc'], instructions='Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. The current date and time is: 05/02/24 14:51:39. ', last_error=None, metadata={}, model='chat4turbo', object='thread.run', required_action=None, started_at=1714679500, status='expired', thread_id='thread_SBWbbdBGwzswzaWXWa5khTWL', tools=[CodeInterpreterTool(type='code_interpreter'), FunctionTool(function=FunctionDefinition(name='get_stock_price', description='Retrieve the latest closing price of a stock using its ticker symbol.', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required': ['symbol']}), type='function

In [24]:
process_prompt("What is my best and worst investment?")

processing ...
Run(id='run_NVKc3eitZqHCeJ5BqIn2BGLR', assistant_id='asst_GLYqs7FKGhO6eSbAqLEcOGVZ', cancelled_at=None, completed_at=None, created_at=1714680100, expires_at=None, failed_at=None, file_ids=['assistant-CgPhDS6uXvZkiaQVRVLLK3jc'], instructions='Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. The current date and time is: 05/02/24 15:01:40. ', last_error=None, metadata={}, model='chat4turbo', object='thread.run', required_action=None, started_at=1714680101, status='expired', thread_id='thread_SBWbbdBGwzswzaWXWa5khTWL', tools=[CodeInterpreterTool(type='code_interpreter'), FunctionTool(function=FunctionDefinition(name='get_stock_price', description='Retrieve the latest closing price of a stock using its ticker symbol.', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required': ['symbol']}), type='function

In [25]:
process_prompt("Chart the realized gain or loss of my investments.")

processing ...
Run(id='run_eQmSOyENq1eJiF2yTo5yvFFZ', assistant_id='asst_GLYqs7FKGhO6eSbAqLEcOGVZ', cancelled_at=None, completed_at=None, created_at=1714680701, expires_at=None, failed_at=None, file_ids=['assistant-CgPhDS6uXvZkiaQVRVLLK3jc'], instructions='Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. The current date and time is: 05/02/24 15:11:41. ', last_error=None, metadata={}, model='chat4turbo', object='thread.run', required_action=None, started_at=1714680702, status='expired', thread_id='thread_SBWbbdBGwzswzaWXWa5khTWL', tools=[CodeInterpreterTool(type='code_interpreter'), FunctionTool(function=FunctionDefinition(name='get_stock_price', description='Retrieve the latest closing price of a stock using its ticker symbol.', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required': ['symbol']}), type='function

In [26]:
client.beta.assistants.delete(assistant.id)
client.beta.threads.delete(thread.id)
for file in assistant_files:
    client.files.delete(file.id)